# 3D Annotation Guide

This guide will help you upload a Point Cloud to Diffgram using the Diffgram SDK, we will then visualize the point cloud in the 3D Annotation tool and draw some cuboids to annotate the point cloud data. Finally we will generate an export JSON for ingestion on any training model you have.

If you want to read the docs and see some UI screenshots you can visit this same guide on our docs: https://diffgram.readme.io/docs/3d-lidar-annotation-guide
# Pre-Requisites


1. A Working Diffgram Installation (either with docker or directly on diffgram.com)
2. An API Key for usage with the SDK
3. A Point Cloud File. (Already Included in the Notebook)
4. Diffgram SDK installed (>=0.6.0)




In [5]:
!pip install diffgram==0.6.0 open3d
!wget https://github.com/mrdoob/three.js/raw/master/examples/models/pcd/binary/Zaghetto.pcd

     |████████████████████████████████| 395.5 MB 30 kB/s 
     |████████████████████████████████| 596 kB 41.0 MB/s 
     |████████████████████████████████| 8.6 MB 44.1 MB/s 
     |████████████████████████████████| 1.0 MB 55.4 MB/s 
     |████████████████████████████████| 396 kB 55.3 MB/s 
     |████████████████████████████████| 428 kB 56.6 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
     |████████████████████████████████| 129 kB 63.6 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 78 kB 7.0 MB/s 
     |████████████████████████████████| 69 kB 7.0 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1
  Attempting uninstall: jsonschema
    Found existing 

--2021-12-14 21:23:07--  https://github.com/mrdoob/three.js/raw/master/examples/models/pcd/binary/Zaghetto.pcd
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mrdoob/three.js/master/examples/models/pcd/binary/Zaghetto.pcd [following]
--2021-12-14 21:23:07--  https://raw.githubusercontent.com/mrdoob/three.js/master/examples/models/pcd/binary/Zaghetto.pcd
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 721096 (704K) [application/octet-stream]
Saving to: ‘Zaghetto.pcd.3’

Zaghetto.pcd.3      100%[===================>] 704.20K  --.-KB/s    in 0.02s   

2021-12-14 21:23:07 (27.9 MB/s) - ‘Zaghetto

# 1. Ingest The Data Using the SDK
For the data ingestion, we will use the Diffgram SDK. The basic idea is to iterate over the points in the point cloud to create a Diffgram File3D object. Here's the sample code to open our test Point cloud file and upload it to Diffgram.

Some notes:

* We are assuming the filename is Zaghetto.pcd as we are sing the PCD file from the pre-requisites section. Feel free to change this to whatever path you point cloud file is.
* We are using open3d library to read the point cloud, but feel free to use whatever method you have to get the points data. It can be from a lidar sensor directly, a .obj file, glb file, etc. As long as the correct parameters are provided to the add_point method. You should be good to go.


Let's start by importing some librarie and creating a small function to read a PCD file.

In [6]:
from diffgram.file.file_3d import File3D
from diffgram.core.core import Project
import time
import open3d as o3d
import numpy as np 

# For reading the PCD file
def read_pcd_o3d(file_name):
    pcd = o3d.io.read_point_cloud(file_name)
    out_arr = np.asarray(pcd.points)
    return out_arr



Now we can go ahead and setup our Diffgram project. Make sure to replace the credentials with the ones on your project. Check https://diffgram.readme.io/reference/authentication for more info on how to get your credentials.

In [7]:
project = Project(project_string_id = "your_project_name",
                  client_id = "your_client_id",
                  client_secret = "your_client_secret")



INFO - 2021-12-14 21:23:25,556 - directory - Using Default Dataset ID 6897


No we can go ahead and upload the PCD file, we will read each point and append it to our File3D object. After all point are added, we call the upload() function to send it to Diffgram.

In [8]:
points_arr = read_pcd_o3d('Zaghetto.pcd')

diffgram_3d_file = File3D(client = project, name = 'face{}.pcd'.format(time.time()))

for i in range(0, len(points_arr)):
  point = points_arr[i]
  diffgram_3d_file.add_point(
    x = point[0],
    y = point[1],
    z = point[2],
  )
diffgram_3d_file.upload()

True

That's it! You can now visit your project at diffgram.com to see the uploaded PCD file and start annotating it. 